In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader, PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from kiwipiepy import Kiwi

# reorder
from langchain_community.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
kiwi = Kiwi()

In [3]:
def kiwi_tokenize(docs):
    return [token.form for token in kiwi.tokenize(docs)]

In [4]:
loaders = [
    PDFPlumberLoader("data/Adaptive_RAG.pdf"),
    PDFPlumberLoader("data/Naive_RAG.pdf"),
    PDFPlumberLoader('data/RAPTOR_RAG.pdf'),
    PDFPlumberLoader('data/Self_RAG.pdf')
]

docs = []

for loader in loaders:
    docs.extend(loader.load())

In [5]:
len(docs)

87

In [6]:
# Ensemble Retriever
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

embedding = OpenAIEmbeddings(model="text-embedding-3-small")

documents = text_splitter.split_documents(docs)

chroma_vector_store = Chroma.from_documents(documents, embedding)
chroma_retriever = chroma_vector_store.as_retriever()

kiwi_vector_store = BM25Retriever.from_documents(documents, preprocess_func=kiwi_tokenize)

ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, kiwi_vector_store],
    weights=[0.5, 0.5],
    search_kwargs={"k": 10}
    #search_type="mmr"
)

In [7]:
# MultiQUery Retriever
from langchain.retrievers.multi_query import MultiQueryRetriever
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever = ensemble_retriever,
    llm=llm
)

In [8]:
# Reorder
def reorder_documents(docs):
    # 재정렬
    reordering = LongContextReorder()
    reordered_docs = reordering.transform_documents(docs)
    return reordered_docs

In [9]:
# prompt
template = """
    당신은 주어진 문서에 대해서 QA를 해주는 assistant bot입니다.
    주어진 문서를 이용해서 답변해 주세요
    만약 주어진 질문에 대해서 모른다면, 모른다고 답변해 주세요
    문서의 출처와 페이지 넘버를 작성해주세요
    한국어로 답변해주세요

    #Example Format:
    (brief summary of the answer)
    (table)
    (detailed answer to the question)

    **출처**
    - (page source and page number)

    # Question:
    {question}

    # Context:
    {context}

    # Answer
"""
prompt = PromptTemplate.from_template(template, input_variable=["question", "context"])

In [10]:
# chain
chain = ({"context":ensemble_retriever| RunnableLambda(reorder_documents), "question": RunnablePassthrough()}
         |prompt
         |llm
         |StrOutputParser()
         )

In [11]:
print(chain.invoke("AdaptiveRAG에 대해서 설명해 주세요"))

Adaptive-RAG는 적응형 검색 기반 생성 모델로, 복잡한 질문에 대해 효과적으로 대응하기 위해 설계되었습니다. 이 모델은 질문의 복잡성을 분류하는 오라클 분류기를 사용하여, 단순한 질문과 복잡한 질문에 대해 각각 다른 접근 방식을 적용합니다. Adaptive-RAG는 LLM(대형 언어 모델)과 검색기를 반복적으로 활용하여 최적의 답변을 도출합니다.

| 특징                | 설명                                                                 |
|-------------------|--------------------------------------------------------------------|
| 모델 유형           | 적응형 검색 기반 생성 모델                                             |
| 질문 처리 방식      | 질문의 복잡성에 따라 다른 전략을 적용                                   |
| 사용 기술           | LLM과 검색기를 반복적으로 활용하여 답변 도출                          |
| 성능               | 복잡한 질문에 대해 효과적이며, 효율성 또한 높음                        |

Adaptive-RAG는 단순한 질문에 대해서는 LLM의 패러미터 지식을 활용하고, 복잡한 질문에 대해서는 검색 기반 접근 방식을 통해 답변을 생성합니다. 이 모델은 다양한 질문 유형에 적응할 수 있는 능력을 가지고 있으며, 실험 결과에서 다른 모델들보다 더 효과적이고 효율적인 성능을 보여주었습니다.

**출처**
- Adaptive_RAG.pdf, 페이지 6, 7, 13


In [12]:
print(chain.invoke("Self-RAG에 대해서 설명해 주세요"))

Self-RAG(자기 반영적 검색 증강 생성)는 대형 언어 모델(LLM)의 품질과 사실성을 향상시키기 위해 설계된 프레임워크입니다. 이 방법은 검색과 자기 반영을 통해 LLM이 필요에 따라 텍스트를 생성하고, 생성된 결과를 비판적으로 평가할 수 있도록 학습합니다. Self-RAG는 일반적인 RAG 접근 방식과 달리, 검색된 정보의 완전한 지원을 보장하며, LLM의 창의성과 다재다능성을 해치지 않으면서도 사실적 정확성을 높이는 데 중점을 둡니다.

| Self-RAG의 주요 특징 |
|---------------------|
| 검색과 자기 반영을 통한 품질 향상 |
| 필요에 따라 텍스트 생성 |
| 생성된 결과에 대한 비판적 평가 |
| 사실적 정확성 유지 |
| LLM의 창의성과 다재다능성 보존 |

Self-RAG는 LLM이 작업 입력에 따라 자신의 생성 과정을 반영하고, 필요할 경우 검색된 구문을 기반으로 텍스트를 생성하도록 훈련됩니다. 이 과정에서 LLM은 '반영 토큰'을 생성하여 검색의 필요성을 신호하거나 출력의 관련성, 지원 또는 완전성을 확인합니다. 이러한 방식은 LLM이 제공된 구문에서 사실을 활용하고 따르도록 명시적으로 훈련되지 않은 기존 RAG 접근 방식의 한계를 극복합니다.

**출처**
- Self-RAG.pdf, 페이지 2, 0, 1, 9, 15
